In [2]:
include(joinpath(dirname(Base.active_project()), "ClassImport.jl"))
using HDF5, JSON
using .ClassImport: GreedyRegression, Integrate, ReportModel, PickSubdomains, EnvelopePol, FUNCTION_PATH, HDF5_PATH, JSON_PATH

p::Array{Float64} = Float64[]
t::Array{Float64} = Float64[]
u::Array{Float64} = Float64[]
v::Array{Float64} = Float64[]
x::Array{Float64} = Float64[]
y::Array{Float64} = Float64[]
Bx::Array{Float64} = Float64[]
By::Array{Float64} = Float64[]

h5open(joinpath(HDF5_PATH, "implicit", "data-RK1.hdf5"), "r") do file
    global p = read(file, "result")["value"]["p"]["value"]
    global t = read(file, "result")["value"]["t"]["value"]
    global u = read(file, "result")["value"]["u"]["value"]
    global v = read(file, "result")["value"]["v"]["value"]
    global x = read(file, "result")["value"]["x"]["value"]
    global y = read(file, "result")["value"]["y"]["value"]
    global Bx = read(file, "result")["value"]["Bx"]["value"]
    global By = read(file, "result")["value"]["By"]["value"]
end

y

1×64 Matrix{Float64}:
 0.0  0.0981748  0.19635  0.294524  …  5.89049  5.98866  6.08684  6.18501

In [3]:
nw::Int = 128 # number of domains we integrate over 
dof::Int = 1 # scalars have one degree of freedom
dim::Int = 3 # how many dimensions does our data have?
env_pow::Int = 4 # weight is (1-x^2)^power
size_vec::Vector{Int} = [16,16,16] # how many gridpoints should we use per integration?
buffer::Int = 0 # Don't use points this close to boundary

pol::Matrix{Float64} = EnvelopePol.envelope_pol(env_pow, dim)

size_of_data::Vector{Int} = [size(u,1), size(u,2), size(u,3)]
seed::Int = 4
corners::Matrix{Int} = PickSubdomains.pick_subdomains(size_of_data, size_vec, buffer, nw, seed)

grid::Vector{Vector{Float64}} = [vec(y), vec(x), vec(t)]
corners

3×128 Matrix{Int64}:
 42  45  33  25  20  16  11  34  27  …  32  31  42  16  47  16  13  17  35
 41  31   5  10   2  43  42  25  42     42  36   3  29   7  28  14   3   9
 58  73  47  33  43  34  27  79  79     46  70  64  71   2  64  12  16  48

In [4]:
using Statistics

G::Matrix{Float64} = zeros(Float64, dof*nw, 0)
labels::Vector{String} = String[]
scales::Vector{Float64} = Float64[]

function add_library_term(label::AbstractString, data::AbstractArray, derivs::AbstractVector{Int64}, scale::AbstractFloat)
    col::Vector{Float64} = Integrate.poly_integrate(data, derivs, grid, corners, size_vec, pol)

    global G = hcat(G, col)
    push!(labels, label)
    push!(scales, scale)
end

# flow terms
add_library_term("∂u/∂x", u, [2], 1.)
add_library_term("∂v/∂x", v, [2], 1.)
add_library_term("∂u/∂y", u, [1], 1.)
add_library_term("∂v/∂y", v, [1], 1.)
add_library_term("∂u/∂t", u, [3], 1.)
add_library_term("∂v/∂t", v, [3], 1.)
add_library_term("∂u^2/∂x", u .* u, [2], 1.)
add_library_term("∂uv/∂x", u .* v, [2], 1.)
add_library_term("∂uv/∂y", u .* v, [1], 1.)
add_library_term("∂v^2/∂y", v .* v, [1], 1.)

# flow terms second derivatives
add_library_term("∂^2u/∂x^2", u, [2,2], 1.)
add_library_term("∂^2u/∂y^2", u, [1,1], 1.)
add_library_term("∂^2v/∂x^2", v, [2,2], 1.)
add_library_term("∂^2v/∂y^2", v, [1,1], 1.)

# pressure terms
add_library_term("∂P/∂x", p, [2], 1.)
add_library_term("∂P/∂y", p, [1], 1.)

# B field terms
add_library_term("∂Bx^2/∂x", Bx .* Bx, [2], 1.)
add_library_term("∂Bx^2/∂y", Bx .* Bx, [1], 1.)
add_library_term("∂By^2/∂x", By .* By, [2], 1.)
add_library_term("∂By^2/∂y", By .* By, [1], 1.)
add_library_term("∂BxBy/∂x", Bx .* By, [2], 1.)
add_library_term("∂BxBy/∂y", Bx .* By, [1], 1.)
add_library_term("∂Bx/∂t", Bx, [3], 1.)
add_library_term("∂By/∂t", By, [3], 1.)

# B field second derivatives
add_library_term("∂^2Bx/∂x^2", Bx, [2, 2], 1.)
add_library_term("∂^2By/∂x^2", By, [2, 2], 1.)
add_library_term("∂^2Bx/∂y^2", Bx, [1, 1], 1.)
add_library_term("∂^2By/∂y^2", By, [1, 1], 1.)

# weird terms
add_library_term("∂uBy/∂x", u .* By, [2], 1.)
add_library_term("∂vBx/∂x", v .* Bx, [2], 1.)
add_library_term("∂uBy/∂y", u .* By, [1], 1.)
add_library_term("∂vBx/∂y", v .* Bx, [1], 1.)

# error
add_library_term("h∂^2Bx/∂t^2", Bx, [3,3], 0.015625)
add_library_term("h∂^2By/∂t^2", By, [3,3], 0.015625)
add_library_term("h∂^2u/∂t^2", u, [3,3], 0.015625)
add_library_term("h∂^2v/∂t^2", v, [3,3], 0.015625)

norm_vec::Vector{Float64} = Integrate.poly_integrate(0*u .+ 1, Int64[], grid, corners, size_vec, pol)
G ./= norm_vec
G ./= scales'

G

128×36 Matrix{Float64}:
 -0.0125664     0.000192278   0.0143686    …   0.00168544   -0.00136136
 -0.00414779    0.00411403    0.0196366        0.000430172   0.00130793
 -0.00543429    0.00724515   -0.0171801       -0.000354616  -0.00250155
 -0.00646279    0.0226108    -0.0304786        0.00342197   -0.00278061
  0.000733214  -0.00503795   -0.007177        -0.00124625   -0.000525728
  0.0154983     0.00405252   -0.0247469    …  -0.0110344    -0.00383325
  0.0213395     0.000609433  -0.0177752       -0.00273475   -0.00844569
  0.000258034  -0.00869844    0.0152237        0.00137258    0.00113416
  0.0114203     0.0049352    -0.0063962        0.00156847   -0.000510909
 -0.000748728  -0.00114758    0.0166904        0.00029991   -0.000642479
 -0.00416793    0.00258777   -0.00535846   …  -0.000945591   0.000936008
 -0.000558051  -0.00368228    0.032607         0.00427663    0.00389972
 -0.00500989   -0.00261442    0.000118985     -0.00191362    0.00402456
  ⋮                                 

In [5]:
using LinearAlgebra

gamma::Float64 = 2

for i::Int = 1:5
    cs::Matrix{Float64}, residuals::Vector{Float64}, _ = GreedyRegression.greedy_regression(G)
    println(findall((residuals[1:end-1]) ./ (residuals[2:end]) .> gamma))

    k::Int, _, _, eqs::String = ReportModel.report_identified_model(cs, residuals, scales, labels, gamma)

    col_norms::Vector{Float64} = [norm(G[:, j] .* cs[j, k]) for j = 1:size(G, 2)]
    kill::Int = argmax(col_norms)
    G = G[:, setdiff(1:end, kill)]
    labels = labels[setdiff(1:end, kill)]
    scales = scales[setdiff(1:end, kill)]

    println(eqs)
end

[4]
∂Bx/∂t - 0.016∂^2Bx/∂x^2 - 0.016∂^2Bx/∂y^2 - ∂uBy/∂y + ∂vBx/∂y = 0
[4]
∂By/∂t - 0.016∂^2By/∂x^2 - 0.016∂^2By/∂y^2 + ∂uBy/∂x - ∂vBx/∂x = 0
[1]
∂u/∂x + ∂v/∂y = 0
[7]
∂u/∂t + ∂u^2/∂x + ∂uv/∂y - 0.016∂^2u/∂x^2 - 0.016∂^2u/∂y^2 + ∂P/∂x - ∂Bx^2/∂x - ∂BxBy/∂y = 0
[7]
∂v/∂t + ∂uv/∂x + ∂v^2/∂y - 0.016∂^2v/∂x^2 - 0.016∂^2v/∂y^2 + 0.992∂P/∂y - ∂By^2/∂y - ∂BxBy/∂x = 0
